In [0]:
import numpy as np
import pandas as pd
import datetime
import pickle
import matplotlib.pyplot as plt

GET DATA

In [0]:

data = pickle.load(open("/content/drive/My Drive/0000/data.pkl",'rb'))

class get_data():
    
    def __init__(self, data):
        self.c_data = data
        self.data = self.clean_data()
        
    def clean_data(self):
        for d in range(len(data)-1, -1, -1):
            if data[d]['event_type'] != 'encounter':
                data.pop(d)
        data_clean = data
        return data_clean
   
    def time_in_range(self, start, end, xs,xe):
        #Return true if x is in the range [start, end]
        if start <= end:
            return start<= xs and xe <= end
        else:
            
            return start<= xs or xe <= end

    def max_time(self): 

        # Initialize maximum time 
        # print(self.data[0])
        max_time1 = self.data[0]['time']
        for d in self.data:
            if d['time'] > max_time1: 
                #print(type(d['time']))
                max_time1 = d['time']
        return max_time1

    def min_time(self):
        min_time1 = self.data[0]['time']
        for d in self.data:
            if d['time'] < max_time1: 
                #print(type(d['time']))
                min_time1 = d['time']
        return min_time1    
        
    def time_slice(self):
        # The Time Span is from days chosen by user to current date time
        # The date-time of encounter + duration should fall in the time slice
        # self.clean_data()
        time_slice_start = self.max_time() - datetime.timedelta(days = int(input("Enter Number of Days of Simulation")))
        # print(time_slice_start)
        time_slice_end = self.max_time()
        # print(time_slice_end)
        timestep = int(input("Enter timestep"))
        
        dfinal=[]
        interactions={}
        cdata = self.data
        while(time_slice_start < time_slice_end):
            dlist=[]
            for d in cdata:
                interactions={}
                time_interact_start = d['time']
                if (d['time'] >= time_slice_start) and (d['time'] < (time_slice_start+datetime.timedelta(hours=timestep))):
                    interactions['lambda']=d['payload']['unobserved'].get('signal_strength')
                    interactions['human_id']=d['human_id']
                    interactions['other_human_id']=d['payload']['unobserved']['human2_id']
                    interactions['human1_isinfected']=d['payload']['unobserved']['human1']['is_infected']
                    interactions['human2_isinfected']=d['payload']['unobserved']['human2']['is_infected']
                    dlist.append(interactions)
            dfinal.append(dlist)  
            time_slice_start+=datetime.timedelta(hours = int(timestep))
        # print(len(dfinal))
        # print(dfinal)        
        return dfinal

    def time_slice1(self):
        # The Time Span is from days chosen by user to current date time
        # The date-time of encounter + duration should fall in the time slice
        # self.clean_data()
        print("SIMULATOR STARTS ON: ",self.min_time())
        print(" AND ENDS ON: ",self.max_time())

        print("INPUT START DATE : ")
        year = int(input('Enter a year'))
        month = int(input('Enter a month'))
        day = int(input('Enter a day'))
        date1 = datetime.date(year, month, day)
        time_slice_start = date1

        print("INPUT END DATE : ")
        year = int(input('Enter a year'))
        month = int(input('Enter a month'))
        day = int(input('Enter a day'))
        date2 = datetime.date(year, month, day)
        time_slice_end = date2

        timestep = int(input("Enter timestep"))
        
        dfinal=[]
        interactions={}
        cdata = self.data
        while(time_slice_start < time_slice_end):
            dlist=[]
            for d in cdata:
                interactions={}
                time_interact_start = d['time']
                if (d['time'] >= time_slice_start) and (d['time'] < (time_slice_start+datetime.timedelta(hours=timestep))):
                    interactions['lambda']=d['payload']['unobserved'].get('signal_strength')
                    interactions['human_id']=d['human_id']
                    interactions['other_human_id']=d['payload']['unobserved']['human2_id']
                    interactions['human1_isinfected']=d['payload']['unobserved']['human1']['is_infected']
                    interactions['human2_isinfected']=d['payload']['unobserved']['human2']['is_infected']
                    dlist.append(interactions)
            dfinal.append(dlist)  
            time_slice_start+=datetime.timedelta(hours = int(timestep))
        # print(len(dfinal))
        # print(dfinal)        
        return dfinal    

In [9]:
gd = get_data(data)
sir_data = gd.time_slice()

#gd = get_data(data)
#sir_data = gd.time_slice1()

Enter Number of Days of Simulation14
Enter timestep1


MODEL OUTPUTS : INFECTION PROBABILITY

In [0]:
STATES = ["S", "I"]

def get_infection_probas(states, transmissions):
    """
    - states[i] = state of i
    - transmissions = array/list of i, j, lambda_ij
    - infection_probas[i]  = 1 - prod_{j: state==I} [1 - lambda_ij]
    """
    print("states:",states)
    print("transmissions", transmissions)
    #   infected = (states == 1)
    #   print("infected:",infected)
    N = len(states)
    infection_probas = np.zeros(N)
 


    for i in range(N):
        rates = np.array([rate for i0,j,rate in transmissions
                            if states[i][0]==i0 and states[i][1] == 1])
        infection_probas[i] = 1 - np.prod(1 - rates)
    print("infection_probas:", infection_probas)
    return infection_probas 


def propagate(current_states, infection_probas):
    """
    - current_states[i] = state of i
    - infection_probas[i]  = proba that i get infected (if susceptible)
    - recover_probas[i] = proba that i recovers (if infected)
    """
    next_states = [[None]*2]*len(current_states)
    
    
#     next_states = np.zeros_like(current_states)
    for i in range(len(current_states)):
        if (current_states[i][1] == 0):
            infected = np.random.rand() < infection_probas[i]
            current_states[i][1] = 1 if infected else 0
#         elif (current_states[i][1] == 1):
#             recovered = np.random.rand() < recover_probas[i]
#             current_states[i][1] = 2 if recovered else 1
        else:
            current_states[i][1] = 1
    return current_states


class EpidemicModel():
    def __init__(self, initial_states):
        self.N = len(initial_states)
        self.initial_states = initial_states

    def time_evolution(self, transmissions, print_every):
        """Run the simulation where
        - recover_probas[i] = mu_i time-independent
        - transmissions[t] = list of t, i, j, lambda_ij(t)
        - states[t, i] = state of i at time t
        """
        # initialize states
        T = len(transmissions)
        print("initial_states:",self.initial_states)
        states = [[None]*self.N]*(T + 1)
#         print("state length", states)
        states[0] = self.initial_states
        # iterate over time steps
        for t in range(T):
            if (t % print_every == 0):
                print(f"t = {t} / {T}")
            infection_probas = get_infection_probas(states[t], transmissions[t])
            states[t+1] = propagate(states[t], infection_probas)
        self.states = states
#         self.probas = get_dummies(states)    


class run_model(EpidemicModel):
    def __init__(self, data, initial_states = None):
        self.data = data
        self.transmissions = self.generate_transmissions()
        
        if initial_states is None:
            initial_states = []
            duplicates = []
            for i in range(len(self.data)):
                if len(self.data[i]) != 0:
                    for j in range(len(self.data[i])):
                        initial_state_2 = []
                        initial_state_1 = []
                        if self.data[i][j]['other_human_id'] not in duplicates: 
                            if self.data[i][j]['human2_isinfected'] == True:
                                initial_state_2.append(self.data[i][j]['other_human_id'])
                                initial_state_2.append(1)
                                duplicates.append(self.data[i][j]['other_human_id'])
                            else:
                                initial_state_2.append(self.data[i][j]['other_human_id'])
                                initial_state_2.append(0)
                                duplicates.append(self.data[i][j]['other_human_id'])
                            initial_states.append(initial_state_2)
                        if self.data[i][j]['human_id'] not in duplicates: 
                            if self.data[i][j]['human1_isinfected'] == True:
                                initial_state_1.append(self.data[i][j]['human_id'])
                                initial_state_1.append(1)
                                duplicates.append(self.data[i][j]['human_id'])
                            else:
                                initial_state_1.append(self.data[i][j]['human_id'])
                                initial_state_1.append(0)
                                duplicates.append(self.data[i][j]['human_id'])
                            initial_states.append(initial_state_1)
        super().__init__(initial_states)
        


    def generate_transmissions(self):
        transmissions = []
        for i in range(len(self.data)):
#             print(self.data)
            tf = []
            for j in range(len(self.data[i])):
                if len(self.data[i]) != 0:
                    transmission = (self.data[i][j]['human_id'], self.data[i][j]['other_human_id'], self.data[i][j]['lambda'])
                    tf.append(transmission)
                else:
                    tf.append([])
            transmissions.append(tf)
#         print("transmission", transmissions)
        return transmissions
            
    def get_counts(self):
        st=[]
        for i in range(len(self.states)):
            st.append(self.states[i][1])
        a=0
        b=0
        for i in st:
            if i == 1:
                a+=1
            else:
                b+=1
        counts = {"S":b, "I":a}

        return pd.DataFrame(counts, index= [0])

    def run(self, print_every=1):
        print("Generating transmissions")
        self.generate_transmissions()
        print("Running simulation")
        self.time_evolution(
    self.transmissions, print_every=10
        )


    def test_states(self):
        count = 0
        states_secondlast = [[None for i in range(2)] for j in range(len(self.initial_states)+1)] 
        transmissions_secondlast = [[None for i in range(3)] for j in range(len(self.initial_states)+1)]    

        states_last = [[None for i in range(2)] for j in range(len(self.initial_states)+1)] 
        transmissions_last = [[None for i in range(3)] for j in range(len(self.initial_states)+1)]  
        # Find last seconf lasttransmission of every individual
        tf_last=[]
        tf_secondlast=[]
        for i in range( len(self.data) - 1, -1, -1) :
          if(len(self.data[i])!=0):
            for j in self.data[i]:
              transmission=[]
              if (states_secondlast[j['human_id']][1] == 2):
                if(j['human1_isinfected']):
                  states_secondlast[j['human_id']][1]=1
                else:
                  states_secondlast[j['human_id']][1]=0
                
                # Has been visited once; this the second last transmission
                transmission = (j['human_id'], j['other_human_id'],j['lambda'])
                tf_secondlast.append(transmission)
                
                states_secondlast[j['human_id']][0]=j['human_id']
                
                print("in second transmission", j['human_id'])

              if (states_secondlast[j['human_id']][1] == None):
                print("in none",j['human_id'])

                transmission = (j['human_id'], j['other_human_id'],j['lambda'])
                tf_last.append(transmission)

                states_last[j['human_id']][0]=j['human_id']
                if(j['human1_isinfected']):
                  states_last[j['human_id']][1]=1
                else:
                  states_last[j['human_id']][1]=0
                
                states_secondlast[j['human_id']][1]=2

        print(states_secondlast)

            # This is equivalent to having all encounter in one time step for all ids.
        infection_probas = get_infection_probas(states_secondlast, transmissions_secondlast)
        calculated_states = propagate(states_secondlast, infection_probas)
        
        calculated_states.pop(0)
        states_last.pop(0)
        print(tf_secondlast)
        print(tf_last)
        print("calculated states: ",calculated_states)
        print("final_states: ",states_last)
  
        
        #CHECK
        [i for i, j in zip(calculated_states, states_last) if i == j]
        return (calculated_states, transmissions_last, states_last)




In [40]:
'''
"Each individual j in ∂i(t) infects i with a probability λj→i(t) ie transmissions")
"Each indivisual is some parts susceptible and some parts infected... the probability with which you are likely to be infected. ie infection_probas")
'''
model = run_model(sir_data)
model.run()
calculated_states, tf_last, states_last = model.test_states() 
#print(p) 

Generating transmissions
Running simulation
initial_states: [[1, 1], [50, 1], [3, 1], [24, 1], [76, 1], [93, 1], [59, 1], [98, 1], [51, 1], [96, 1], [57, 1], [5, 1], [89, 1], [25, 1], [61, 1], [20, 0], [71, 1], [63, 1], [11, 1], [62, 0], [73, 1], [4, 1], [80, 1], [85, 1], [22, 1], [6, 1], [7, 1], [8, 0], [48, 1], [78, 1], [9, 1], [10, 1], [12, 1], [13, 1], [14, 1], [15, 1], [16, 1], [17, 1], [18, 1], [19, 1], [52, 1], [84, 1], [37, 1], [77, 1], [82, 1], [90, 1], [46, 1], [44, 1], [41, 1], [55, 1], [23, 1], [60, 1], [42, 1], [70, 1], [66, 1], [34, 1], [72, 1], [88, 1], [40, 1], [74, 1], [87, 1], [29, 1], [81, 1], [49, 1], [99, 1], [21, 1], [68, 1], [53, 1], [91, 1], [30, 1], [79, 1], [45, 1], [56, 1], [32, 1], [95, 1], [36, 1], [35, 1], [67, 1], [94, 1], [27, 1], [92, 1], [75, 1], [38, 1], [54, 1], [65, 1], [86, 1], [83, 1], [26, 1], [31, 1], [47, 1], [69, 1], [97, 1], [28, 1], [58, 1], [64, 1], [33, 0], [2, 1], [43, 0], [39, 1]]
t = 0 / 336
states: [[1, 1], [50, 1], [3, 1], [24, 1], [7

In [0]:
STATES = ["S", "I"]

def get_infection_probas(probas, transmissions, mstates):
    """
    - probas[i,s] = P_s^i(t)
    - transmissions = array/list of i, j, lambda_ij
    - infection_probas[i]  = sum_j lambda_ij P_I^j(t)
    """
    N = probas.shape[0]
    infection_probas = np.zeros(N)
    print("transmissions", transmissions)
    rates= np.array([])
    for i in range(N):
        for i0, j, rate in transmissions:
            for k in range(len(transmissions)):
                if i0 == transmissions[k][0]:
                    j1 = transmissions[k][1]
                    for j2 in range(len(mstates[0])):
                        if mstates[0][j2][0] == j1:
                            rates = np.concatenate([probas[j2, 1]*rate], axis = None)
        srate = 0
        for s in range(len(rates)):
            srate += rates[s]
        infection_probas[i] = srate
    return infection_probas


def propagate(probas, infection_probas):
    """
    - probas[i,s] = P_s^i(t)
    - infection_probas[i]  = proba that i get infected (if susceptible)
    - recover_probas[i] = proba that i recovers (if infected)
    """
    next_probas = np.zeros_like(probas)
    next_probas[:, 0] = probas[:, 0]*(1 - infection_probas)
    next_probas[:, 1] = probas[:, 1]*(1) + probas[:, 0]*infection_probas
    #next_probas[:, 2] = probas[:, 2] + probas[:, 1]*recover_probas
    print("next_probas", next_probas[:, 1])
    return next_probas


class InferenceModel():
    def __init__(self, initial_probas, mstates):
        self.N = len(initial_probas)
        self.initial_probas = initial_probas
        self.mstates = mstates

    def time_evolution(self, transmissions,print_every=10):
        """Run the simulation where
        - recover_probas[i] = mu_i time-independent
        - transmissions[t] = list of t, i, j, lambda_ij(t)
        - probas[t, i, s] = state of i at time t
        """
        # initialize states
        T = len(transmissions)
        probas = np.zeros((T + 1, self.N, 2))
        probas[0] = self.initial_probas
        # iterate over time steps
        for t in range(T):
            if (t % print_every == 0):
                print(f"t = {t} / {T}")
            infection_probas = get_infection_probas(probas[t], transmissions[t], self.mstates)
            probas[t+1] = propagate(probas[t], infection_probas)
        self.probas = probas
        # print("probas", self.probas)
        self.states = probas.argmax(axis=2)
        # print("states", self.states)
        
    def get_counts(self):
        counts = self.probas.sum(axis=1)
        return pd.DataFrame(counts, columns=STATES)

In [0]:
def generate_initial_probas(M, states):
    for i in range(len(states)):
        freqs = [np.mean(states[i][1]==s) for s in [0,1]]
    N = len(states)
    initial_probas = np.broadcast_to(freqs, (N, 2)).copy()
    observations = np.random.choice([states[i][0] for i in range(len(states))], M, replace=False)
    for i in observations:
        for j in range(len(states)):
            if states[j][0] == i:
                s = int(states[j][1])
                initial_probas[j] = np.zeros(2)
                initial_probas[j, s] = 1.
    return initial_probas


def test_next_probas():
    T = len(tf_secondlast)
    probas = np.zeros((T + 1, 99, 2))
    initial_probas1 = generate_initial_probas(M=99, states=calculated_states)
    probas = initial_probas1

    infection_probas1 = get_infection_probas(probas, tf_last, calculated_states)
    new_probas = propagate(probas, infection_probas1)

    print(next_probas)

    

In [43]:

#t_obs = 6
#initial_probas = generate_initial_probas(M=10, states=model.states[t_obs])
#infer = InferenceModel(initial_probas, model.states)
#infer.time_evolution(model.transmissions[t_obs:], print_every=1)

test_next_probas()


transmissions [[None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None

TypeError: ignored

TEST FOR MODEL AND INFERENCE
SINCE WE HAVE 2 OUTPUTS TO TEST INFECTION PROBABBILITY NAD STATES IT IS ONLY FAIR TO HAVE TWO TESTS